#### Training GAT

Model using the graph dataset extracted from the csv file. TO generate csv file, please run the parsing notebook. 

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import dgl
import dgl.nn as dglnn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# ------------------------------
# 1. Load Dataset and Prepare Data
# ------------------------------
df = pd.read_csv("all_circuits_features.csv")
print("\n?? Dataset Overview:")
print(df.info())

label_encoder = LabelEncoder()
df["gate_label"] = label_encoder.fit_transform(df["gate_type"])

feature_columns = [
    "fan_in", "fan_out", "dist_to_output", "is_primary_input", "is_primary_output",
    "is_internal", "is_key_gate", "degree_centrality", "betweenness_centrality",
    "closeness_centrality", "clustering_coefficient", "avg_fan_in_neighbors", "avg_fan_out_neighbors"
]

df = df.dropna(subset=feature_columns)
df[feature_columns] = df[feature_columns].astype(float)

scaler = StandardScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

# ---------------------------
# 2. Build Node and Edge Lists
# ---------------------------
nodes = df["node"].tolist()
node_to_id = {node: i for i, node in enumerate(nodes)}

edges = []
for _, row in df.iterrows():
    node_id = node_to_id[row["node"]]
    potential_sources = df[df["fan_out"] > 0]["node"].tolist()
    num_fan_in = int(row["fan_in"])
    if num_fan_in > 0:
        sources = potential_sources[:num_fan_in]
        for src in sources:
            if src in node_to_id:
                edges.append((node_to_id[src], node_id))

print("\n?? Extracted", len(edges), "edges.")
if len(edges) == 0:
    raise ValueError("No edges found! Check your fan_in values.")

src_nodes, dst_nodes = zip(*edges) if edges else ([], [])
src_tensor = torch.tensor(src_nodes, dtype=torch.int64)
dst_tensor = torch.tensor(dst_nodes, dtype=torch.int64)
valid_edges = (
    (src_tensor >= 0) & (dst_tensor >= 0) &
    (src_tensor < len(nodes)) & (dst_tensor < len(nodes))
)
src_tensor = src_tensor[valid_edges]
dst_tensor = dst_tensor[valid_edges]

# ---------------------------
# 3. Create the DGL Graph
# ---------------------------
graph = dgl.graph((src_tensor, dst_tensor), num_nodes=len(nodes))
graph = dgl.add_self_loop(graph)

graph.ndata['features'] = torch.tensor(df[feature_columns].values, dtype=torch.float32)
graph.ndata['labels'] = torch.tensor(df["gate_label"].values, dtype=torch.long)

nodes_idx = np.arange(len(nodes))
train_idx, test_idx = train_test_split(nodes_idx, test_size=0.2, random_state=42)
train_nid = torch.tensor(train_idx, dtype=torch.long)
test_nid = torch.tensor(test_idx, dtype=torch.long)

# ---------------------------
# 4. Define the GAT Model
# ---------------------------
class GATModel(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats, num_heads1=4, num_heads2=1, attn_drop=0.2, feat_drop=0.2):
        super().__init__()
        # First GAT layer outputs [N, num_heads1, hidden_feats]
        self.gat1 = dglnn.GATConv(
            in_feats, hidden_feats, num_heads=num_heads1,
            feat_drop=feat_drop, attn_drop=attn_drop, activation=nn.ELU()
        )
        # Second GAT layer maps concatenated heads to out_feats (set num_heads2=1 for logits)
        self.gat2 = dglnn.GATConv(
            hidden_feats * num_heads1, out_feats, num_heads=num_heads2,
            feat_drop=feat_drop, attn_drop=attn_drop, activation=None
        )

    def forward(self, g, x):
        x = self.gat1(g, x)                 # [N, H1, D]
        x = x.flatten(1)                    # concat heads -> [N, H1*D]
        x = self.gat2(g, x)                 # [N, H2, C]
        if x.dim() == 3:
            x = x.mean(1)                   # average over heads if H2 > 1 (or keep single head)
        return x

in_feats = len(feature_columns)
hidden_feats = 32
out_feats = len(label_encoder.classes_)
model = GATModel(in_feats, hidden_feats, out_feats, num_heads1=4, num_heads2=1, attn_drop=0.2, feat_drop=0.2)

# ---------------------------
# 5. Full-graph Training Loop
# ---------------------------
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = nn.CrossEntropyLoss()

epochs = 50
for epoch in range(epochs):
    model.train()
    logits = model(graph, graph.ndata['features'])
    loss = loss_fn(logits[train_nid], graph.ndata['labels'][train_nid])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

# ---------------------------
# 6. Evaluation on Full Graph
# ---------------------------
model.eval()
with torch.no_grad():
    logits = model(graph, graph.ndata['features'])
    test_logits = logits[test_nid]
    test_predictions = test_logits.argmax(dim=1)
    orig_accuracy = (test_predictions == graph.ndata['labels'][test_nid]).float().mean().item()

print(f"\n? Test Accuracy on Original Graph: {orig_accuracy * 100:.2f}%")

true_labels_orig = graph.ndata['labels'][test_nid].cpu().numpy()
pred_labels_orig = test_predictions.cpu().numpy()
conf_mat_orig = confusion_matrix(true_labels_orig, pred_labels_orig)

print("\n?? Confusion Matrix (Original Graph):")
print(conf_mat_orig)

print("\nClassification Report (Original Graph):")
print(classification_report(true_labels_orig, pred_labels_orig, target_names=label_encoder.classes_))



?? Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60882 entries, 0 to 60881
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   circuit_name            60882 non-null  object 
 1   node                    60882 non-null  object 
 2   gate_type               60882 non-null  object 
 3   fan_in                  60882 non-null  int64  
 4   fan_out                 60882 non-null  int64  
 5   depth                   60882 non-null  object 
 6   dist_to_output          60882 non-null  int64  
 7   is_primary_input        60882 non-null  int64  
 8   is_primary_output       60882 non-null  int64  
 9   is_internal             60882 non-null  int64  
 10  is_key_gate             60882 non-null  int64  
 11  key_dependency          122 non-null    object 
 12  degree_centrality       60882 non-null  float64
 13  betweenness_centrality  60882 non-null  float64
 14  closeness_centra

/home/rrk307/anaconda3/envs/gnn_circuits/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rrk307/anaconda3/envs/gnn_circuits/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rrk307/anaconda3/envs/gnn_circuits/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

#### Jacobian 

In [2]:
import torch
import numpy as np

print("\n--- Jacobian Computation and Relative Error (GAT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
device = graph.ndata['features'].device
labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid (index array)
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts (selected)
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Helper: f(x) returns logits for node test_idx with its feature vector replaced by x
# --------------------------------
def node_logits_with_replaced_features(x, test_idx):
    # x: [F] tensor on device
    new_features = graph.ndata['features'].clone()
    new_features[test_idx] = x
    out = model(graph, new_features)
    return out[test_idx]  # shape: [num_classes]

# --------------------------------
# 3) Main loop: Jacobian + finite-difference verification
# --------------------------------
class_metrics = {cn: {'jacobian_norms': [], 'relative_errors': []} for cn in class_names}
overall_jacobian_norms = []
overall_relative_errors = []

epsilon = 1e-3  # small perturbation for FD check

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    # Base feature vector for this node
    x0 = graph.ndata['features'][test_idx].clone().detach().to(device).requires_grad_(True)

    # Define f(x) for autograd Jacobian (must not use no_grad)
    def f(x):
        return node_logits_with_replaced_features(x, test_idx)

    # Compute Jacobian J (num_classes x feat_dim)
    J = torch.autograd.functional.jacobian(f, x0)  # shape: [C, F]

    # Frobenius norm of the Jacobian
    jacobian_norm = torch.norm(J, p='fro').item()

    # Finite-difference verification
    delta = epsilon * torch.randn_like(x0)
    predicted_change = J.mv(delta)  # [C]
    f_x0 = f(x0)
    f_x0_perturbed = f(x0 + delta)
    actual_change = f_x0_perturbed - f_x0
    rel_error = (torch.norm(predicted_change - actual_change) /
                 (torch.norm(actual_change) + 1e-8)).item()

    # Store metrics
    class_metrics[class_name]['jacobian_norms'].append(jacobian_norm)
    class_metrics[class_name]['relative_errors'].append(rel_error)
    overall_jacobian_norms.append(jacobian_norm)
    overall_relative_errors.append(rel_error)

# --------------------------------
# 4) Reporting
# --------------------------------
print("\nClass-wise Jacobian Analysis Results:")
header = "{:<12s} {:>30s} {:>35s}".format("Class", "Avg. Jacobian Norm ± Std", "Avg. Relative Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    norms = class_metrics[cn]['jacobian_norms']
    errors = class_metrics[cn]['relative_errors']
    avg_norm = np.mean(norms) if norms else 0.0
    std_norm = np.std(norms) if norms else 0.0
    avg_rel_error = np.mean(errors) if errors else 0.0
    std_rel_error = np.std(errors) if errors else 0.0
    print("{:<12s} {:>15.4f} ± {:<12.4f} {:>15.4e} ± {:<10.4e}".format(
        cn, avg_norm, std_norm, avg_rel_error, std_rel_error
    ))

# Overall aggregates
overall_avg_norm = float(np.mean(overall_jacobian_norms)) if overall_jacobian_norms else 0.0
overall_std_norm = float(np.std(overall_jacobian_norms)) if overall_jacobian_norms else 0.0
overall_avg_rel_error = float(np.mean(overall_relative_errors)) if overall_relative_errors else 0.0
overall_std_rel_error = float(np.std(overall_relative_errors)) if overall_relative_errors else 0.0

print("\nOverall Aggregated Jacobian Analysis Results:")
print("Average Jacobian Norm: {:.4f} ± {:.4f}".format(overall_avg_norm, overall_std_norm))
print("Average Relative Error: {:.4e} ± {:.4e}".format(overall_avg_rel_error, overall_std_rel_error))



--- Jacobian Computation and Relative Error (GAT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Jacobian Analysis Results:
Class              Avg. Jacobian Norm ± Std           Avg. Relative Error ± Std
-------------------------------------------------------------------------------
and                   9.3621 ± 1.6380            2.2930e-04 ± 1.8074e-04
input                 7.5426 ± 0.1753            4.2216e-04 ± 1.9531e-04
nand                  9.0070 ± 1.4154            2.8325e-04 ± 2.3040e-04
nor                   8.4702 ± 0.9996            2.8

#### Local Lipschitz constant

In [3]:
import torch
import numpy as np

print("\n--- Local Lipschitz Constant & Relative Error (GAT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid (index array)
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts (selected)
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Helper: f(x) returns logits for node test_idx with its feature vector replaced by x
# --------------------------------
def node_logits_with_replaced_features(x, test_idx):
    new_features = graph.ndata['features'].clone()
    new_features[test_idx] = x
    out = model(graph, new_features)
    return out[test_idx]  # [num_classes]

# --------------------------------
# 3) Main loop: Local Lipschitz (||J||_2) + finite-difference relative error
# --------------------------------
class_lipschitz = {cn: [] for cn in class_names}
class_rel_errors = {cn: [] for cn in class_names}
overall_lipschitz = []
overall_rel_errors = []

epsilon = 1e-3
trials_per_node = 10

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    # Base feature vector
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)

    # Define scalar-to-vector function for Jacobian
    def f(x):
        return node_logits_with_replaced_features(x, test_idx)

    # Jacobian J: [C, F]
    J = torch.autograd.functional.jacobian(f, x0)

    # Local Lipschitz constant: operator 2-norm (largest singular value)
    L_local = torch.linalg.norm(J, ord=2).item()

    # Finite-difference relative error across multiple random directions
    rel_errors_for_node = []
    f_x0 = f(x0).detach()
    for _ in range(trials_per_node):
        delta = epsilon * torch.randn_like(x0)
        # Predicted change norm via Lipschitz bound
        pred_change_norm = L_local * torch.norm(delta).item()
        # Actual change norm
        f_x0_pert = f(x0 + delta).detach()
        actual_change_norm = torch.norm(f_x0_pert - f_x0).item()
        rel_err = abs(pred_change_norm - actual_change_norm) / (actual_change_norm + 1e-8)
        rel_errors_for_node.append(rel_err)

    avg_rel_error_node = float(np.mean(rel_errors_for_node))

    # Store
    class_lipschitz[class_name].append(L_local)
    class_rel_errors[class_name].append(avg_rel_error_node)
    overall_lipschitz.append(L_local)
    overall_rel_errors.append(avg_rel_error_node)

# --------------------------------
# 4) Reporting
# --------------------------------
print("\nClass-wise Local Lipschitz (with Relative Errors):")
header = "{:<12s} {:>25s} {:>30s}".format("Class", "Avg Lipschitz ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    L_vals = class_lipschitz[cn]
    E_vals = class_rel_errors[cn]
    if L_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(L_vals), np.std(L_vals), np.mean(E_vals), np.std(E_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

if overall_lipschitz:
    print("\nOverall Aggregated:")
    print("Avg Lipschitz Constant: {:.4f} ± {:.4f}".format(np.mean(overall_lipschitz), np.std(overall_lipschitz)))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))
else:
    print("\nOverall Aggregated:")
    print("No samples selected; overall metrics unavailable.")



--- Local Lipschitz Constant & Relative Error (GAT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Local Lipschitz (with Relative Errors):
Class              Avg Lipschitz ± Std           Avg Rel. Error ± Std
---------------------------------------------------------------------
and               6.3116 ± 0.9519          2.0032e+00 ± 5.4596e-01
input             5.4617 ± 0.1419          2.1204e+00 ± 3.6971e-01
nand              6.1662 ± 0.7786          2.0747e+00 ± 6.2590e-01
nor               5.7727 ± 0.5582          1.9881e+00 ± 4.6152e-01
not     

#### Hessian-Based Curvature Measure

In [4]:
import torch
import numpy as np

print("\n--- Hessian-Based Curvature Measure & Relative Error (GAT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Helper to compute Hessian, gradient, and scalar function h at x0
# --------------------------------
def compute_hessian_and_grad_for_sample(test_idx):
    # Base input with gradients
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)

    # Determine predicted class at x0
    with torch.no_grad():
        base_logits = model(graph, graph.ndata['features'])
        pred_class = torch.argmax(base_logits[test_idx])

    # Scalar function: log prob of predicted class
    def h(x):
        new_features = graph.ndata['features'].clone().detach()
        new_features[test_idx] = x
        logits = model(graph, new_features)[test_idx]
        log_probs = torch.log_softmax(logits, dim=0)
        return log_probs[pred_class]

    # Hessian at x0
    H = torch.autograd.functional.hessian(h, x0, create_graph=False, vectorize=False).detach()

    # Gradient at x0
    g = torch.autograd.functional.jacobian(h, x0).detach()

    # Scalar baseline h(x0)
    h0 = h(x0.detach())

    return H, x0.detach(), g, h0, h

# --------------------------------
# 3) Main loop: curvature proxy + relative error
# --------------------------------
class_hessian_eig = {cn: [] for cn in class_names}
class_rel_errors  = {cn: [] for cn in class_names}
overall_hessian_eig = []
overall_rel_errors  = []

epsilon = 5e-3
trials_per_node = 10

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    H, x0, g, h0, h_func = compute_hessian_and_grad_for_sample(test_idx)

    # Largest eigenvalue (lambda_max)
    try:
        eigvals = torch.linalg.eigvalsh(H)
        lambda_max = torch.max(eigvals).item()
    except RuntimeError:
        # Fallback: power iteration
        v = torch.randn_like(x0)
        v = v / (v.norm() + 1e-12)
        for _ in range(20):
            v = H @ v
            v = v / (v.norm() + 1e-12)
        lambda_max = torch.dot(v, H @ v).item()

    # Relative error of second-order approximation
    node_errors = []
    for _ in range(trials_per_node):
        delta = epsilon * torch.randn_like(x0)
        pred_second = 0.5 * torch.dot(delta, H @ delta).item()
        actual_second = (h_func(x0 + delta) - h0 - torch.dot(g, delta)).item()
        rel_error = abs(pred_second - actual_second) / (abs(actual_second) + 1e-8)
        node_errors.append(rel_error)

    avg_rel_error_node = float(np.mean(node_errors))

    # Store
    class_hessian_eig[class_name].append(lambda_max)
    class_rel_errors[class_name].append(avg_rel_error_node)
    overall_hessian_eig.append(lambda_max)
    overall_rel_errors.append(avg_rel_error_node)

# --------------------------------
# 4) Reporting
# --------------------------------
print("\nClass-wise Hessian-Based Curvature (with Relative Errors):")
header = "{:<12s} {:>33s} {:>30s}".format("Class", "Avg. Max Eigenvalue ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    c_vals = class_hessian_eig[cn]
    e_vals = class_rel_errors[cn]
    if c_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(c_vals), np.std(c_vals), np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:>10s} {:>15s} {:>10s}".format(cn, "-", "-", "-", "-"))

if overall_hessian_eig:
    print("\nOverall Aggregated Hessian-Based Curvature:")
    print("Avg Max Eigenvalue: {:.4f} ± {:.4f}".format(np.mean(overall_hessian_eig), np.std(overall_hessian_eig)))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))
else:
    print("\nOverall Aggregated Hessian-Based Curvature:")
    print("No samples selected; overall metrics unavailable.")



--- Hessian-Based Curvature Measure & Relative Error (GAT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Hessian-Based Curvature (with Relative Errors):
Class                Avg. Max Eigenvalue ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------------
and               0.0552 ± 0.0743          1.1623e-01 ± 3.2371e-01
input             0.0000 ± 0.0000          5.3810e-01 ± 1.7081e-01
nand              0.1739 ± 0.1180          1.1373e-01 ± 2.6615e-01
nor               0.0473 ± 0.0987          5

#### Prediction Margin

In [5]:
import torch
import numpy as np

print("\n--- Prediction Margin & Relative Error (GAT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Main loop: Prediction margin + relative error
# --------------------------------
class_margin_vals = {cn: [] for cn in class_names}
class_rel_errors  = {cn: [] for cn in class_names}
overall_margin_vals = []
overall_rel_errors  = []

epsilon = 1e-5  # small perturbation for verification

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    # Original logits
    with torch.no_grad():
        logits = model(graph, graph.ndata['features'])[test_idx]

    pred_class = int(torch.argmax(logits))
    pred_logit = logits[pred_class].item()

    # Second max logit
    other_logits = logits.clone()
    other_logits[pred_class] = -float('inf')
    second_max = other_logits.max().item()

    margin = pred_logit - second_max

    # Relative error verification
    perturbed_feats = graph.ndata['features'].clone()
    perturb_vec = torch.randn_like(perturbed_feats[test_idx]) * epsilon
    perturbed_feats[test_idx] += perturb_vec

    with torch.no_grad():
        logits_pert = model(graph, perturbed_feats)[test_idx]
    pred_logit_pert = logits_pert[pred_class].item()
    other_logits_pert = logits_pert.clone()
    other_logits_pert[pred_class] = -float('inf')
    second_max_pert = other_logits_pert.max().item()
    margin_pert = pred_logit_pert - second_max_pert

    rel_err = abs(margin - margin_pert) / (abs(margin_pert) + 1e-12)

    # Store
    class_margin_vals[class_name].append(margin)
    class_rel_errors[class_name].append(rel_err)
    overall_margin_vals.append(margin)
    overall_rel_errors.append(rel_err)

# --------------------------------
# 3) Reporting
# --------------------------------
print("\nClass-wise Prediction Margin (with Relative Error):")
header = "{:<12s} {:>33s} {:>30s}".format("Class", "Avg. Margin ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    m_vals = class_margin_vals[cn]
    e_vals = class_rel_errors[cn]
    if m_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(m_vals), np.std(m_vals),
            np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

if overall_margin_vals:
    print("\nOverall Aggregated Prediction Margin:")
    print("Avg Margin: {:.4f} ± {:.4f}".format(np.mean(overall_margin_vals), np.std(overall_margin_vals)))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))
else:
    print("\nOverall Aggregated Prediction Margin:")
    print("No samples selected; overall metrics unavailable.")



--- Prediction Margin & Relative Error (GAT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Prediction Margin (with Relative Error):
Class                        Avg. Margin ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------------
and               3.1969 ± 1.4619          8.9702e-06 ± 9.0874e-06
input            13.8249 ± 0.9520          1.9818e-06 ± 1.5289e-06
nand              1.0233 ± 0.6257          2.1029e-04 ± 1.8450e-03
nor               1.6374 ± 0.7790          3.8585e-05 ± 1.9952e-0

#### Adversarial Robustness Radius

In [7]:
import torch
import numpy as np

print("\n--- Adversarial Robustness Radius & Relative Error (GAT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Prediction wrapper for a node (replace its feature vector with x)
# --------------------------------
def f_for_sample(x, test_idx):
    """
    Given an input x (modified feature vector for the test node),
    returns the output (logit vector) for that node.
    """
    new_features = graph.ndata['features'].clone().detach()
    new_features[test_idx] = x
    with torch.no_grad():
        out = model(graph, new_features)
    return out[test_idx]

# --------------------------------
# 3) Adversarial radius along random directions + binary search
# --------------------------------
def adversarial_radius_for_sample(
    test_idx,
    initial_epsilon=1e-3,
    growth_factor=1.2,
    max_epsilon=10.0,
    bs_iters=10,
    num_trials=10,
):
    """
    Minimal perturbation norm required to change the prediction, estimated by:
      - Sampling random directions d (||d||=1)
      - Expanding epsilon multiplicatively until the prediction flips (or cap)
      - Binary-searching in the bracket to refine the boundary
    Returns the minimum over 'num_trials' sampled directions.
    """
    x0 = graph.ndata['features'][test_idx].clone().detach()

    with torch.no_grad():
        base_out = model(graph, graph.ndata['features'])
        y0 = int(torch.argmax(base_out[test_idx]).item())

    def is_same(x):
        out = f_for_sample(x, test_idx)
        return int(torch.argmax(out).item()) == y0

    radii = []
    for _ in range(num_trials):
        d = torch.randn_like(x0)
        d = d / (torch.norm(d) + 1e-12)

        epsilon = initial_epsilon
        # Expand until flip or cap
        while epsilon < max_epsilon and is_same(x0 + epsilon * d):
            epsilon *= growth_factor

        if epsilon >= max_epsilon:
            candidate = max_epsilon
        else:
            # Binary search in [epsilon/growth_factor, epsilon]
            low = epsilon / growth_factor
            high = epsilon
            for _ in range(bs_iters):
                mid = 0.5 * (low + high)
                if is_same(x0 + mid * d):
                    low = mid
                else:
                    high = mid
            candidate = high
        radii.append(candidate)

    return float(min(radii))

# --------------------------------
# 4) Relative error verification (re-estimate with different config)
# --------------------------------
def adversarial_radius_relerr(test_idx):
    r1 = adversarial_radius_for_sample(
        test_idx,
        initial_epsilon=1e-3,
        growth_factor=1.2,
        max_epsilon=10.0,
        bs_iters=10,
        num_trials=10,
    )
    r2 = adversarial_radius_for_sample(
        test_idx,
        initial_epsilon=1e-3,
        growth_factor=1.3,  # different growth factor / search iters
        max_epsilon=10.0,
        bs_iters=12,
        num_trials=10,
    )
    rel_err = abs(r1 - r2) / (abs(r2) + 1e-12)
    return r1, rel_err

# --------------------------------
# 5) Main loop: compute ARR + relative error per class
# --------------------------------
class_adv_radius = {cn: [] for cn in class_names}
class_rel_errors  = {cn: [] for cn in class_names}
overall_adv_radius_vals = []
overall_rel_errors      = []

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    radius, rel_err = adversarial_radius_relerr(test_idx)

    class_adv_radius[class_name].append(radius)
    class_rel_errors[class_name].append(rel_err)
    overall_adv_radius_vals.append(radius)
    overall_rel_errors.append(rel_err)

# --------------------------------
# 6) Reporting
# --------------------------------
print("\nClass-wise Adversarial Robustness Radius (with Relative Error):")
header = "{:<12s} {:>27s} {:>30s}".format("Class", "Avg. Radius ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    r_vals = class_adv_radius[cn]
    e_vals = class_rel_errors[cn]
    if r_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(r_vals), np.std(r_vals),
            np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

if overall_adv_radius_vals:
    print("\nOverall Aggregated Adversarial Robustness Radius:")
    print("Avg Radius: {:.4f} ± {:.4f}".format(np.mean(overall_adv_radius_vals), np.std(overall_adv_radius_vals)))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))
else:
    print("\nOverall Aggregated Adversarial Robustness Radius:")
    print("No samples selected; overall metrics unavailable.")



--- Adversarial Robustness Radius & Relative Error (GAT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Adversarial Robustness Radius (with Relative Error):
Class                  Avg. Radius ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------
and               2.1802 ± 2.0720          3.3623e-01 ± 3.2424e-01
input             7.7854 ± 1.5368          2.0797e-01 ± 1.8350e-01
nand              1.3549 ± 1.3504          3.7618e-01 ± 5.7107e-01
nor               1.9357 ± 1.5046          2.5689e-01

#### Stability Under Input Noise

In [6]:
import torch
import numpy as np

print("\n--- Stability Under Input Noise & Relative Error (GAT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Helper: stability under Gaussian noise
# --------------------------------
def stability_for_sample(test_idx, sigma, num_samples):
    """
    Average L2 change in logits between clean and noisy versions of the node.
    """
    x0 = graph.ndata['features'][test_idx].clone().detach()
    with torch.no_grad():
        f_orig = model(graph, graph.ndata['features'])[test_idx]

    diffs = []
    for _ in range(num_samples):
        noise = sigma * torch.randn_like(x0)
        x_noisy = x0 + noise
        new_feats = graph.ndata['features'].clone().detach()
        new_feats[test_idx] = x_noisy
        with torch.no_grad():
            f_noisy = model(graph, new_feats)[test_idx]
        diffs.append(torch.norm(f_noisy - f_orig).item())
    return float(np.mean(diffs))

# --------------------------------
# 3) Main loop: stability + relative error
# --------------------------------
class_stability = {cn: [] for cn in class_names}
class_rel_errors = {cn: [] for cn in class_names}
overall_stability_vals = []
overall_rel_errors = []

sigma = 0.01
num_noise_samples = 20
relerr_resamples = 5

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    stab_val = stability_for_sample(test_idx, sigma, num_noise_samples)

    # Relative error check: re-estimate stability with fresh noise
    re_vals = [stability_for_sample(test_idx, sigma, num_noise_samples)
               for _ in range(relerr_resamples)]
    avg_reval = float(np.mean(re_vals))
    rel_err = abs(stab_val - avg_reval) / (abs(avg_reval) + 1e-12)

    class_stability[class_name].append(stab_val)
    class_rel_errors[class_name].append(rel_err)
    overall_stability_vals.append(stab_val)
    overall_rel_errors.append(rel_err)

# --------------------------------
# 4) Reporting
# --------------------------------
print("\nClass-wise Stability Under Input Noise (with Relative Error):")
header = "{:<12s} {:>33s} {:>30s}".format("Class", "Avg. Stability ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    s_vals = class_stability[cn]
    e_vals = class_rel_errors[cn]
    if s_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(s_vals), np.std(s_vals),
            np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

if overall_stability_vals:
    print("\nOverall Aggregated Stability Under Input Noise:")
    print("Avg Stability: {:.4f} ± {:.4f}".format(np.mean(overall_stability_vals), np.std(overall_stability_vals)))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))
else:
    print("\nOverall Aggregated Stability Under Input Noise:")
    print("No samples selected; overall metrics unavailable.")



--- Stability Under Input Noise & Relative Error (GAT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Stability Under Input Noise (with Relative Error):
Class                     Avg. Stability ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------------
and               0.0861 ± 0.0173          7.8425e-02 ± 6.2327e-02
input             0.0700 ± 0.0073          9.0830e-02 ± 5.9467e-02
nand              0.0837 ± 0.0159          9.2659e-02 ± 6.2658e-02
nor               0.0777 ± 0.0117          7.